# Prior distributions

One important aspect of Bayesian inference has not yet been discussed in this tutorial: [prior distributions](https://en.wikipedia.org/wiki/Prior_probability). In Bayesian statistics, one has to provide probability (density) values for every possible parameter value *before* taking into account the data at hand. This prior distribution thus reflects all *prior* knowledge of the system that is to be investigated. In the case no prior knowledge is available, a *non-informative* prior in the form of the so-called [Jeffreys prior](https://en.wikipedia.org/wiki/Jeffreys_prior) allows to minimize the effect of the prior on the results. The next two sub-sections discuss how one can set custom prior distributions for the parameters of the observation model and for hyper-parameters in a hyper-study or change-point study.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt # plotting
import seaborn as sns           # nicer plots
sns.set_style('whitegrid')      # plot styling

import numpy as np
import bayesloop as bl

# prepare study for coal mining data
S = bl.Study()
S.loadExampleData()
S.setObservationModel(bl.om.Poisson())
S.setGrid([[0, 6, 1000]])

C:\Anaconda\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


+ Created new study.
+ Successfully imported example data.
+ Observation model: Poisson. Parameter(s): ['lambda']
+ Boundaries: [[0, 6]]
+ Grid size: [1000]


## Parameter priors
*bayesloop* employs a forward-backward algorithm that is based on [Hidden Markov models](http://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf). This inference algorithm iteratively produces a parameter distribution for each time step, but it has to start these iterations from a specified probability distribution - the parameter prior. All built-in observation models already have a pre-defined prior, stored in the attribute `prior`. Here, the prior distribution is stored as a lambda-function that takes as many arguments as there are parameters in the observation model. The prior distributions can be looked up directly within `observationModels.py`. For the `Poisson` model discussed in this tutorial, the prior distribution is defined as
```
lambda x: np.sqrt(1./x)
```
corresponding to the non-informative Jeffreys prior, $p(\lambda) \propto 1/\sqrt{\lambda}$. This type of prior can also be determined automatically for arbitrary user-defined observation models, see [here](customobservationmodels.html#Sympy.stats-random-variables).

To change the predefined prior of a given observation model, the `Study` class provides the `setPrior` method. There are two possible ways of defining a parameter prior in *bayesloop*: One is to provide a function that takes exactly as many arguments as there are parameters in the defined observation model. The following example defines a flat (uniform) prior for the `Poisson` model with only one parameter, using a lambda function:

In [2]:
S.setPrior(lambda x: 1)

+ Set custom prior: <lambda>


Note that all custom prior distributions are re-normalized by *bayesloop*, so they do not need to be passed in a normalized form.

The second option is based on the [SymPy](http://www.sympy.org/en/index.html) module that introduces symbolic mathematics to Python. Its sub-module [sympy.stats](http://docs.sympy.org/dev/modules/stats.html) covers a wide range of discrete and continuous random variables. `setPrior` also accepts a list of `sympy.stats` random variables, one for each parameter (if there is only one parameter, the list can be omitted). The multiplicative joint probability density of these random variables is then used as the prior distribution. The following example defines an exponential prior for the `Poisson` model, favoring small values of the rate parameter: 

In [3]:
import sympy.stats
S.setPrior(sympy.stats.Exponential('expon', 1))

+ Set custom prior: exp(-x)


Note that one needs to assign a name to each `sympy.stats` variable. In this case, the output of *bayesloop* shows the mathematical formula that defines the prior. This is possible because of the symbolic representation of the prior by `SymPy`.

## Hyper-parameter priors
As shown before, [hyper-studies](hyperstudy.html) and [change-point studies](changepointstudy.html) can be used to determine the full distribution of hyper-parameters (the parameters of the transition model). As for the *"regular"* parameters of the observation model, one might have prior knowledge about the values of certain hyper-parameters that can be included into the study to refine the resulting distribution of these hyper-parameters. Hyper-parameter priors can be defined just as regular priors, either by an arbitrary function or by a list of `sympy.stats` random variables.

In the case of a hyper-study, the number of arguments for the function or the number of random variables, respectively, is given by the number of hyper-parameters that are included in the hyper-grid, defined by the `setHyperGrid` method. The following example revisits the two break-point-model from [here](changepointstudy.html#Analyzing-structural-breaks-in-time-series-models) and a linear decrease with a varying slope as a hyper-parameter. 
```
import sympy.stats

S = bl.HyperStudy()
S.loadExampleData()
S.setObservationModel(bl.om.Poisson())

T = bl.tm.SerialTransitionModel(bl.tm.Static(),
                                15,
                                bl.tm.Deterministic(lambda t, slope=-0.3: t*slope),
                                30,
                                bl.tm.Static()
                                )
S.setTransitionModel(T)

S.setHyperGrid([['slope', -2.0, 0.0, 30]])
S.setHyperPrior(sympy.stats.Normal('slope_prior', -0.2, 0.4))

S.fit()
```
In this example, there is only one hyper-parameter - slope - for which we define a Normal distribution as the prior distribution, which is centered around the value -0.2 with a standard deviation of 0.4.

However, if we instead analyze the model above using a `ChangepointStudy`, the two break-points defined in the transition model are treated as hyper-parameters, too. Therefore, the prior distribution has to take three parameters into account. Using a lambda function, we can easily employ a flat (uniform) prior for the break-points, and again use a Normal prior for the slope hyper-parameter:
```
S = bl.ChangepointStudy()

...

S.setHyperGrid([['slope', -2.0, 0.0, 30]])
S.setHyperPrior(lambda b1, b2, slope: np.exp(-0.5*((slope + 0.2)/(2*0.4))**2)/0.4)

S.fit()
```
Note that the lambda function includes the break-points as arguments (`b1` and `b2`), but the result does not depend on them, i.e. the prior is flat regarding these two hyper-parameters. All custom hyper-parameter prior distributions are re-normalized by *bayesloop*, so they do not need to be passed in a normalized form.